In [1]:
import numpy as np
import itertools

import cirq
from cirq import ops, Circuit
from cirq.ops import CZ, H, CNOT, X, Y, Z, SWAP
from cirq.google import ExpWGate

from cirq.contrib.rearrange import separate
from cirq.contrib.rearrange.separate import convert_circuit, non_clifford_half, clifford_half

from cirq.contrib.rearrange.axis import *
from cirq.contrib.rearrange.clifford_pauli_gate import CliffordPauliGate
from cirq.contrib.rearrange.interaction_gate import InteractionGate
from cirq.contrib.rearrange.pauli_string import PauliString
from cirq.contrib.rearrange.non_clifford_gate import NonCliffordGate
from cirq.contrib.rearrange import optimize_pauli_strings as opt_strings
from cirq.contrib.rearrange.optimize_pauli_strings import (
    circuit_to_strings, strings_to_circuit, optimize_pauli_strings, get_ordered_qubits, display_strings)

from cirq.testing import assert_allclose_up_to_global_phase

from cirq.contrib.rearrange.separate_test import toffoli_circuit, qft_circuit

In [2]:
import pickle

def saveCircuit(fName):
    with open(fName, 'wb') as f:
        pickle.dump(circuit, f)
def loadCircuit(fName):
    with open(fName, 'rb') as f:
        return pickle.load(f)

In [3]:
from IPython.display import display

def _ops_to_matrix(*ops, draw=True):
    c = cirq.Circuit.from_ops(*ops)
    if draw: display(c)
    m = c.to_unitary_matrix()
    return m.round(5)

In [4]:
q0, q1, q2 = (cirq.NamedQubit('q{}'.format(i)) for i in range(3))

In [5]:
def make_circuit():
    return toffoli_circuit()
    #return loadCircuit('fermionSwap8.pkl')
    #return loadCircuit('fermionSplit8.pkl')

In [6]:
make_circuit()

q0: ─────────────────────────────────────────@──────────────────────────────────────────────────@────────────────────────@───Z^0.25────@───────────
                                             │                                                  │                        │             │
q1: ───────────────@─────────────@───X───@───X───@───X───@────────────@─────────────@───X───@───X───@───X───@───Z^0.25───X───Z^-0.25───X───────────
                   │             │   │   │       │   │   │            │             │   │   │       │   │   │
q2: ───Y^0.5───X───X───Z^-0.25───X───@───X───────X───@───X───Z^0.25───X───Z^-0.25───X───@───X───────X───@───X───Z^0.25─────────────────Y^0.5───X───

In [17]:
large_circ = make_circuit()
#cirq.google.ConvertToXmonGates().optimize_circuit(large_circ)
cirq.google.MergeInteractions(allow_partial_czs=False).optimize_circuit(large_circ)
#cirq.google.MergeRotations().optimize_circuit(large_circ)
display(large_circ)
cirq.google.MergeInteractions(allow_partial_czs=False).optimize_circuit(large_circ)
cirq.google.MergeRotations().optimize_circuit(large_circ)
cirq.circuits.DropNegligible().optimize_circuit(large_circ)
cirq.circuits.DropEmptyMoments().optimize_circuit(large_circ)
large_circ

[[ 0.65328148  0.27059805  0.65328148  0.27059805]
 [ 0.27059805  0.65328148 -0.27059805 -0.65328148]
 [ 0.65328148 -0.27059805 -0.65328148  0.27059805]
 [ 0.27059805 -0.65328148  0.27059805 -0.65328148]]
[-0.92387953+0.38268343j  0.92387953-0.38268343j  0.38268343+0.92387953j
  0.38268343+0.92387953j]
[[-0.70710678  0.          0.          0.70710678]
 [ 0.          0.70710678  0.70710678  0.        ]
 [ 0.70710678  0.          0.          0.70710678]
 [ 0.         -0.70710678  0.70710678  0.        ]]
[[ 0.34186054  0.67941408 -0.19595027 -0.61897607]
 [ 0.61897607  0.19595027  0.67941408  0.34186054]
 [-0.6571923   0.64922922  0.28018104  0.26095647]
 [-0.26095647 -0.28018104  0.64922922 -0.6571923 ]]
[ 1.0000000e+00-7.32394011e-18j  1.0000000e+00+1.73941487e-18j
 -5.7987430e-18+1.00000000e+00j -6.2242356e-18+1.00000000e+00j]
[[ 0.34186054  0.61897607 -0.26095647  0.6571923 ]
 [ 0.67941408  0.19595027 -0.28018104 -0.64922922]
 [-0.19595027  0.67941408  0.64922922 -0.28018104]
 [-0.6

q0: ──────────────────────────────────────────────────────────────────────────────────────────────────────@───────────────────────────────────────────────────────────────────────────────────────────────────────@─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────Y^-0.5───────────────@───X^-0.25───@───Y^0.5───Z^0.25───────────
                                                                                                          │                                                                                                       │                                                                                                                                                      │             │
q1: ───────────────X────────Y^-0.5───@───Z^-0.5───Y^0.5───X^0.5───@───Z^-0.5───X^-0.5───X^0.5───Z^0.25────X───W(0.75)^-0.719───Z^-0.75────────Y^-0.5────────────@───Z^-0.5───Y^0.5───X^-0.781───Z^0.75────────────X───X^-0.5───Z^0.5────Y^-0.5────────────@───Z^-0.5───Y^0.5───X^0.5───@───Z^-0.5───X^-0.5───@───Z^0.5───Y^-0.5───Z^0.75────────────Y^0.5────Z───H───────Z─────────────Z───H───────Y^0.5────Z───────
                                     │                            │                                                                                             │                                                                                         │                            │                     │
q2: ───Y^0.5───X───Y^-0.5───Y^-0.5───Z───Z^-0.5───Y^0.5───X^0.5───Z───Z^-0.5───X^-0.5───────────W(0.25)────────────────────────W(0.71)^-0.5───Z^0.79───Y^-0.5───Z───Z^-0.5───Y^0.5──────────────Y^-0.5───Z^0.71────────────────X^-0.5───Z^0.5────Y^-0.5───Z───Z^-0.5───Y^0.5───X^0.5───Z───Z^-0.5───X^-0.5───Z───Z^0.5────────────Y^-0.5───Z^0.75────────────────Y^0.5──────────────────────────────────────────X───

[[-0.65328148 -0.27059805 -0.65328148 -0.27059805]
 [-0.27059805 -0.65328148  0.27059805  0.65328148]
 [ 0.65328148 -0.27059805 -0.65328148  0.27059805]
 [ 0.27059805 -0.65328148  0.27059805 -0.65328148]]
[-0.92387953+0.38268343j  0.92387953-0.38268343j  0.38268343+0.92387953j
  0.38268343+0.92387953j]
[[ 0.00000000e+00 -1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00  0.00000000e+00  6.62303530e-17  0.00000000e+00]
 [ 5.55111512e-17  0.00000000e+00 -1.00000000e+00 -2.22044605e-16]
 [ 5.55111512e-17  0.00000000e+00  2.11937130e-16 -1.00000000e+00]]
[[ 0.27548053 -0.59737392  0.11251359  0.74471174]
 [-0.28221616 -0.12438505  0.94125032 -0.13758717]
 [-0.13406756 -0.79184333 -0.22545771 -0.55152412]
 [-0.90911517 -0.02563031 -0.2248496   0.34970751]]
[-0.70710678+0.70710678j  0.70710678-0.70710678j  0.70710678-0.70710678j
  0.70710678-0.70710678j]
[[ 0.58955936 -0.60513218 -0.07446453 -0.5298017 ]
 [-0.10418357 -0.18524233 -0.94990423  0.22915718]
 [ 0.27435328 -0.4660

q0: ─────────────────────────────────────────────────────────@───────────────────────────────────────────────────────────@─────────────────────────────────────────────────────────────────────W(0.644)^0.5──────Z^-0.144───@───X^-0.25───@───Y^-0.5────Z^0.394───
                                                             │                                                           │                                                                                                  │             │
q1: ────────────Y^0.5───@───Y^0.5───@───W(0.875)─────────────X───W(0.25)^0.5───Z^0.531───@───W(0.781)^-0.5────Z^-0.531───X───W(0.155)^0.129───Z^-0.317───@───Y^0.5───@───Y^-0.5───Z^-0.5───@───W(0.77)^-0.411────Z^0.177────Z─────────────Z───Z^0.915─────────────
                        │           │                                                    │                                                               │           │                     │
q2: ───Y^-0.5───────────Z───Y^0.5───Z───Y^0.5──────Z^-0.21───────────────────────────────Z───W(0.806)^0.244───Z^-0.936───────────────────────────────────Z───Y^0.5───Z───Y^-0.5───Z^-0.5───Z───W(0.375)^-0.445───Z^-0.438─────────────────────────────────────────

In [68]:
# 174?, 176?, 147!

In [8]:
#large_circ = make_circuit()
#display(large_circ)
#circuit = convert_circuit(large_circ, pre_optimize=True)
circuit = convert_circuit(large_circ, pre_optimize=False)
#circuit = convert_circuit(large_circ, pre_optimize=True)
#circuit = convert_circuit(large_circ, pre_optimize=True)
#circuit = convert_circuit(large_circ, pre_optimize=True)
c_left0 = non_clifford_half(circuit)
c_right0 = clifford_half(circuit)
pauli_strings, c_left_right = optimize_pauli_strings(circuit_to_strings(c_left0), max_steps=100)
#opt_strings.remove_duplicate_strings(pauli_strings)
#opt_strings.remove_negligible_strings(pauli_strings)
c_left = strings_to_circuit(pauli_strings)
display(c_left)
c_right = c_left_right + c_right0
c_right

Remove duplicate
Reduced length from 38 to 25
Remove negligible
Reduced length from 25 to 25
Move clifford out
Reduced length from 25 to 21
Remove duplicate
Reduced length from 21 to 18
Remove negligible
Reduced length from 18 to 18
Move clifford out
Reduced length from 18 to 17
Remove duplicate
Reduced length from 17 to 17
Remove negligible
Reduced length from 17 to 17
Move clifford out
Reduced length from 17 to 17
Final length 17


q0: ───────────────────────────[Z]^-0.25───[Z]^-0.59───[Z]^1.25────────────────[Z]^-0.25────[Z]^-0.409───[Z]^0.458───[Z]^-1.92───────────────────────────
                               │           │           │                                    │            │           │
q1: ───[Z]^-1.16───[Z]^-0.75───[Z]─────────[X]─────────┼──────────[Z]^0.0432────────────────[X]──────────[Y]─────────[X]─────────[X]^0.25────────────────
                   │           │                       │
q2: ───────────────[X]─────────[X]─────────────────────[X]─────────────────────[X]^-0.837───[Y]^0.158────[X]^1.7─────[Y]^0.455───[Z]^0.626───[Y]^0.136───

q0: ───────────────────────────────────────────────X───Z───X────────────────────X^0.5───Z────────────Z───nX^0.5──────────────────────────────────────────────────────────────────────Z───────────────────────Z───X^0.5────────────────────────────────────────────────────────────────────────────────Z───Z───nZ^0.5───nX^0.5───Z^0.5───
                                                   │       │                            │            │                                                                               │                       │                                                                                        │   │
q1: ────────────X^0.5───nZ^0.5───nX^0.5────────────┼───────┼───────────nY^0.5───────────X───nZ^0.5───X───Y^0.5────nX^0.5───Z───nZ^0.5───X^0.5───Z^0.5───Z^0.5───Z───nZ^0.5───Z^0.5───X───X^0.5───Z───X^0.5───X───────────Z───nZ^0.5───nX^0.5───Z^0.5───Z───Z───nZ^0.5───nX^0.5───Z^0.5───nZ^0.5───Z───Z───Z────────────X────────────────
                                                   │       │                                                               │                                    │                                │                       │                                 │                                      │
q2: ───nZ^0.5───X^0.5───nZ^0.5───nX^0.5───nY^0.5───Z───────Z───Y^0.5──────────────────────────────────────────────nX^0.5───Z───nZ^0.5───X^0.5───Z^0.5───Z^0.5───Z───X^0.5────────────────────────Z───────────────────────Z───nZ^0.5───nX^0.5───Z^0.5───Z───Z───nZ^0.5───nX^0.5───Z^0.5───nZ^0.5───Z─────────────────────────────────────

In [9]:
display_strings(pauli_strings, get_ordered_qubits(c_right))

Total length: 17
 . Z .  -1.16189138
 . Z X  -0.75000000
 Z Z X  -0.25000000
 Z X .  -0.58974958
 Z . X  1.25000000
 . Z .  0.04321703
 . . X  -0.83742394
 Z . .  -0.25000000
 . . Y  0.15788154
 Z X .  -0.40946087
 . . X  1.70451875
 Z Y .  0.45849943
 . . Y  0.45532116
 Z X .  -1.92398450
 . . Z  0.62607065
 . X .  0.25000000
 . . Y  0.13551215


In [10]:
assert_allclose_up_to_global_phase(large_circ.to_unitary_matrix(),
                                   (c_left + c_right).to_unitary_matrix(),
                                   atol=1e-7)

In [72]:
c_right

q0: ───────────────────────────────────────────────────────────────────────────Z────────────────────────────────────Z───nZ^0.5───nX^0.5───Z^0.5────────────────────────────────────────────────────────────────────────────────────Z───────────────Z───nZ^0.5───X^0.5───Z^0.5───
                                                                               │                                    │                                                                                                              │               │
q1: ───nZ^0.5───nX^0.5───Z^0.5───Z───Z───nZ^0.5───X^0.5───Z^0.5───Z────────────X───────────────X^0.5───Z───nX^0.5───X────────────X────────────────Z───nZ^0.5───X^0.5───Z^0.5───Z───nZ^0.5───nX^0.5───Z^0.5───nZ^0.5───Z───X^0.5────Z───────────────Z───nZ^0.5───X───────Z^0.5───
                                     │                            │                                    │                                          │                            │                                      │
q2: ───nZ^0.5───X^0.5────Z^0.5───Z───Z───nZ^0.5───X^0.5───Z^0.5───Z───nZ^0.5───X^0.5───Z^0.5───────────Z───X──────────────────────────────────────Z───nZ^0.5───X^0.5───Z^0.5───Z───nZ^0.5───nX^0.5───Z^0.5───nZ^0.5───Z───nZ^0.5───Z^0.5───Z^0.5────────────────────────────────

In [73]:
#pss_long = pauli_strings
#pss_short = pauli_strings

In [74]:
qubits = get_ordered_qubits(c_right)
for (ps1, ht1), (ps2, ht2) in itertools.zip_longest(pss_short, pss_long, fillvalue=(PauliString({}), 0)):
    print('{: .8f}  '.format(ht1), end='')
    for qubit in qubits:
        axis = ps1.get_axis(qubit)
        print('{!s:>2}'.format(axis.abs() if not isinstance(axis, IdentAxis) else '.'), end='')
    print('   ', end='')
    for qubit in qubits:
        axis = ps2.get_axis(qubit)
        print('{!s:>2}'.format(axis.abs() if not isinstance(axis, IdentAxis) else '.'), end='')
    print('  {: .8f}'.format(ht2))

-0.25000000   . Z X    . Z .  -0.12765556
-0.25000000   Z Z X    . Z X   0.75000000
 0.75000000   . Z .    Z Z X  -0.25000000
 0.25000000   Z . X    Z X X  -0.53082420
-0.75000000   . . X    Z . X   0.25000000
-0.25000000   Z Z .    . Z .   0.28972436
 0.25000000   Z . .    . . X  -0.12584751
 0.00000000   . . .    Z . .   0.75000000
 0.00000000   . . .    . Z Z   0.47061260
 0.00000000   . . .    . . X  -0.56539301
 0.00000000   . . .    Z X X  -0.45002144
 0.00000000   . . .    . Z Z  -0.88293876
 0.00000000   . . .    Z Y X   0.21218740
 0.00000000   . . .    . Z Y   0.44851554
 0.00000000   . . .    Z X X   0.58294157
 0.00000000   . . .    . Z Z   0.54998890
 0.00000000   . . .    . X X   0.25000000


In [60]:
qubits

(NamedQubit('q0'), NamedQubit('q1'), NamedQubit('q2'))

In [118]:
qubits
def axis_sort_key(axis):
    if isinstance(axis, Axis):
        return axis.axis_i % 3
    return 6
    
def string_sort_key(ps1):
    return tuple((ps1.get_axis(q) for q in qubits))

In [33]:
#i = 0
#while i < len(pauli_strings):
#    ps = pauli_strings[i][0]
#    for j in reversed(range(i+1, len(pauli_strings))):
#        ps2 = pauli_strings[j][0]
#        if not ps.commutes_with(ps2):
#            break
#        if ps == ps2:
#            pauli_strings[i][1] += pauli_strings[j][1]
#            pauli_strings.pop(j)
#    i += 1

In [12]:
class Connectivity:
    def connected(self, q0: ops.QubitId, q1: ops.QubitId):
        raise NotImplementedError

class FullConnectivity(Connectivity):
    def connected(self, q0: ops.QubitId, q1: ops.QubitId):
        return True

class DeviceConnectivity(Connectivity):
    def connected(self, q0: ops.QubitId, q1: ops.QubitId):
        if isinstance(q0, type(q1)):
            return q1.is_adjacent(q0)
        elif isinstance(q1, type(q0)):
            return q0.is_adjacent(q1)
        else:
            raise RuntimeError('Different qubit types')